In [1]:
# ── Mount Drive (Colab) ─────────────────────────────────────────────────────────
from google.colab import drive
drive.mount('/content/drive')

# ── Paths (edit if your folder name differs) ───────────────────────────────────
ROOT = "/content/drive/MyDrive/celeba_dataset"
SRC_DIR = f"{ROOT}/Celebrity Image Subsets"    # contains images__<ID>_ folders
COMP_ROOT = f"{ROOT}/CompositeDataset"         # will be created

IMAGES_DIR = f"{COMP_ROOT}/images"
LABELS_DIR = f"{COMP_ROOT}/labels"
META_DIR   = f"{COMP_ROOT}/meta"

# ── Create folders ─────────────────────────────────────────────────────────────
import os, shutil, random, json, math, glob
from pathlib import Path

for p in [COMP_ROOT, IMAGES_DIR, LABELS_DIR, META_DIR]:
    os.makedirs(p, exist_ok=True)
for split in ["train","val","test"]:
    os.makedirs(f"{IMAGES_DIR}/{split}", exist_ok=True)
    os.makedirs(f"{LABELS_DIR}/{split}", exist_ok=True)

# ── Install deps ───────────────────────────────────────────────────────────────
!pip -q install ultralytics==8.3.34 opencv-python pillow numpy tqdm

import cv2
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
from tqdm.auto import tqdm
random.seed(42)
np.random.seed(42)


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 kB 9.9 MB/s eta 0:00:00


In [2]:
# ── Parse celeb IDs from folder names images__<ID>_ ────────────────────────────
folders = sorted([p for p in glob.glob(f"{SRC_DIR}/images__*_") if os.path.isdir(p)])
def parse_id(path):
    # expects .../images__<ID>_/
    base = os.path.basename(path.strip("/"))
    # split on double underscores and trailing underscore
    try:
        return base.split("__")[1].rstrip("_")
    except Exception:
        return None

raw_ids = [parse_id(p) for p in folders]
celeb_ids = [cid for cid in raw_ids if cid is not None]
assert len(celeb_ids) >= 2, "No celebrity folders found. Check naming pattern."

# Ensure unique & stable order (sorted as strings keeps IDs like 3401 intact)
celeb_ids = sorted(list(set(celeb_ids)), key=lambda x: int(x))
num_classes = len(celeb_ids)
print(f"Found {num_classes} celebrity IDs.")

# ── Build id_map.csv (idx → celeb_id) ─────────────────────────────────────────
import csv
with open(f"{META_DIR}/id_map.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["idx","celeb_id"])
    for idx, cid in enumerate(celeb_ids):
        writer.writerow([idx, cid])

# Helper: map celeb_id -> class_idx
id_to_idx = {cid: i for i, cid in enumerate(celeb_ids)}


Found 45 celebrity IDs.


In [3]:
GRID_ROWS, GRID_COLS = 2, 3     # 3×2 grid (6 cells)
CANVAS = (640, 640)
CELL_W = CANVAS[0] // GRID_COLS
CELL_H = CANVAS[1] // GRID_ROWS
GUTTER = 10                     # px gap between cells (keeps boxes separated)

N_BASE = 1200
SPLIT_COUNTS = {"train": 840, "val": 180, "test": 180}
assert sum(SPLIT_COUNTS.values()) == N_BASE

# Gather image lists per celeb_id
id_to_images = {}
valid_exts = {".jpg",".jpeg",".png",".bmp"}
for cid in celeb_ids:
    folder = f"{SRC_DIR}/images__{cid}_"
    imgs = []
    for ext in valid_exts:
        imgs.extend(glob.glob(f"{folder}/**/*{ext}", recursive=True))
    imgs = [p for p in imgs if os.path.isfile(p)]
    if len(imgs) == 0:
        print(f"Warning: no images for celeb {cid}")
    id_to_images[cid] = imgs

# Round-robin sampler to keep balance across 45 celebs
from itertools import cycle
rr_ids = []
repeat = math.ceil((N_BASE * GRID_ROWS * GRID_COLS) / len(celeb_ids))  # total slots / num IDs
balanced_list = celeb_ids * repeat
random.shuffle(balanced_list)
rr = cycle(balanced_list)


In [4]:
def fit_within(img: Image.Image, cell_w, cell_h, keep_aspect=True):
    """Resize PIL image to fit within cell while keeping aspect; returns resized image."""
    if keep_aspect:
        img = img.copy()
        img.thumbnail((cell_w - GUTTER, cell_h - GUTTER), Image.LANCZOS)
        return img
    else:
        return img.resize((cell_w - GUTTER, cell_h - GUTTER), Image.LANCZOS)

def cell_origin(col, row, cell_w, cell_h, gutter=0):
    x0 = col*cell_w + gutter//2
    y0 = row*cell_h + gutter//2
    return x0, y0

def paste_center(canvas: Image.Image, tile: Image.Image, x0, y0, cell_w, cell_h):
    """Paste tile centered within the cell at (x0,y0). Returns (xmin,ymin,xmax,ymax) on canvas."""
    tw, th = tile.size
    cx = x0 + cell_w//2
    cy = y0 + cell_h//2
    xmin = int(cx - tw//2)
    ymin = int(cy - th//2)
    xmin = max(xmin, x0)
    ymin = max(ymin, y0)
    xmax = min(xmin + tw, x0 + cell_w - 1)
    ymax = min(ymin + th, y0 + cell_h - 1)
    canvas.paste(tile, (xmin, ymin))
    return xmin, ymin, xmax, ymax

def xyxy_to_yolo(xmin, ymin, xmax, ymax, W, H):
    w = max(1, xmax - xmin + 1)
    h = max(1, ymax - ymin + 1)
    cx = xmin + w/2
    cy = ymin + h/2
    return cx/W, cy/H, w/W, h/H

def rand_pick_image(cid):
    imgs = id_to_images.get(cid, [])
    if not imgs:
        return None
    return random.choice(imgs)


In [5]:
def next_six_distinct(rr_iter, pool):
    """Return 6 distinct celeb IDs with available images."""
    chosen = set()
    attempt = 0
    while len(chosen) < 6 and attempt < 1000:
        c = next(rr_iter)
        if c in pool and id_to_images.get(c):
            chosen.add(c)
        attempt += 1
    return list(chosen)

counter = {"train":0, "val":0, "test":0}
index = 0

for split in ["train","val","test"]:
    target = SPLIT_COUNTS[split]
    pbar = tqdm(total=target, desc=f"Building {split} composites")
    built = 0
    while built < target:
        # choose 6 distinct celeb IDs
        cids = next_six_distinct(rr, celeb_ids)
        if len(cids) < 6:
            continue

        canvas = Image.new("RGB", CANVAS, (245,245,245))
        labels = []

        # place tiles row-major
        ok = True
        used = set()
        for r in range(GRID_ROWS):
            for c in range(GRID_COLS):
                cid = cids[r*GRID_COLS + c]
                if cid in used:
                    ok = False; break
                used.add(cid)

                src_path = rand_pick_image(cid)
                if src_path is None:
                    ok = False; break

                try:
                    tile = Image.open(src_path).convert("RGB")
                except:
                    ok = False; break

                tile = fit_within(tile, CELL_W, CELL_H, keep_aspect=True)
                x0,y0 = cell_origin(c, r, CELL_W, CELL_H, GUTTER)
                xmin,ymin,xmax,ymax = paste_center(canvas, tile, x0, y0, CELL_W, CELL_H)
                cx,cy,w,h = xyxy_to_yolo(xmin,ymin,xmax,ymax, CANVAS[0], CANVAS[1])

                class_idx = id_to_idx[cid]
                labels.append((class_idx, cx, cy, w, h))
            if not ok: break

        if not ok or len(labels) != GRID_ROWS*GRID_COLS:
            continue

        stem = f"comp_{index:06d}"
        img_out = f"{IMAGES_DIR}/{split}/{stem}.jpg"
        lab_out = f"{LABELS_DIR}/{split}/{stem}.txt"

        canvas.save(img_out, quality=92)

        with open(lab_out, "w") as f:
            for (cls,cx,cy,w,h) in labels:
                f.write(f"{cls} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")

        built += 1
        index += 1
        pbar.update(1)
    pbar.close()


Building train composites:   0%|          | 0/840 [00:00<?, ?it/s]

Building val composites:   0%|          | 0/180 [00:00<?, ?it/s]

Building test composites:   0%|          | 0/180 [00:00<?, ?it/s]

In [6]:
AUG_TRAIN = True

def augment_photometric(pil_img):
    # mild brightness & contrast jitter, occasional blur
    br = ImageEnhance.Brightness(pil_img).enhance( np.random.uniform(0.9, 1.1) )
    ct = ImageEnhance.Contrast(br).enhance( np.random.uniform(0.9, 1.1) )
    if np.random.rand() < 0.25:
        ct = ct.filter(ImageFilter.GaussianBlur(radius=np.random.uniform(0.2, 0.8)))
    return ct

def flip_labels_horiz(yolo_lines):
    out = []
    for line in yolo_lines:
        cls, cx, cy, w, h = line
        cx_flipped = 1.0 - cx
        out.append((cls, cx_flipped, cy, w, h))
    return out

if AUG_TRAIN:
    train_imgs = sorted(glob.glob(f"{IMAGES_DIR}/train/*.jpg"))
    for img_path in tqdm(train_imgs, desc="Augmenting train (+1x)"):
        stem = Path(img_path).stem
        lab_path = f"{LABELS_DIR}/train/{stem}.txt"
        if not os.path.exists(lab_path):
            continue

        # read labels
        yolo_lines = []
        with open(lab_path, "r") as f:
            for ln in f:
                toks = ln.strip().split()
                cls = int(toks[0]); cx,cy,w,h = map(float, toks[1:5])
                yolo_lines.append((cls,cx,cy,w,h))

        # 50% flip horizontally
        img = Image.open(img_path).convert("RGB")
        do_flip = np.random.rand() < 0.5
        if do_flip:
            img_aug = img.transpose(Image.FLIP_LEFT_RIGHT)
            labels_aug = flip_labels_horiz(yolo_lines)
        else:
            img_aug = img.copy()
            labels_aug = yolo_lines

        # photometric
        img_aug = augment_photometric(img_aug)

        # save with new stem
        new_stem = stem + "_aug1"
        img_out = f"{IMAGES_DIR}/train/{new_stem}.jpg"
        lab_out = f"{LABELS_DIR}/train/{new_stem}.txt"

        img_aug.save(img_out, quality=92)
        with open(lab_out, "w") as f:
            for (cls,cx,cy,w,h) in labels_aug:
                f.write(f"{cls} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")


Augmenting train (+1x):   0%|          | 0/840 [00:00<?, ?it/s]

In [7]:
data_yaml = {
    "path": COMP_ROOT,
    "train": "images/train",
    "val":   "images/val",
    "test":  "images/test",
    "nc":    num_classes,
    "names": [str(cid) for cid in celeb_ids]  # <- true celeb IDs as strings
}
import yaml
with open(f"{COMP_ROOT}/data.yaml", "w") as f:
    yaml.safe_dump(data_yaml, f, sort_keys=False)

# Record build meta for reproducibility
build_log = {
    "grid": {"rows": GRID_ROWS, "cols": GRID_COLS, "canvas": CANVAS, "gutter": GUTTER},
    "counts": {"base_total": N_BASE, **SPLIT_COUNTS},
    "aug": {"train_plus_one": bool(AUG_TRAIN), "ops": ["flip(0.5)", "brightness/contrast", "blur(0.25)"]},
    "classes": num_classes,
    "note": "Label class_idx follows meta/id_map.csv; names[] are true celeb IDs."
}
with open(f"{META_DIR}/build_log.json","w") as f:
    json.dump(build_log, f, indent=2)


In [8]:
from ultralytics import YOLO

data_path = f"{COMP_ROOT}/data.yaml"  # points to images/ and labels/

# Start with yolov8n or yolov8s for speed; you can try 'm' later
model = YOLO("yolov8n.pt")
model.train(
    data=data_path,
    imgsz=640,
    epochs=80,
    batch=16,
    device=0,       # auto if GPU present
    project=f"{COMP_ROOT}/runs",
    name="yolov8n_grid3x2"
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 39.5MB/s]


New https://pypi.org/project/ultralytics/8.3.205 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.34 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/celeba_dataset/CompositeDataset/data.yaml, epochs=80, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=/content/drive/MyDrive/celeba_dataset/CompositeDataset/runs, name=yolov8n_grid3x2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualiz

100%|██████████| 755k/755k [00:00<00:00, 7.88MB/s]


Overriding model.yaml nc=80 with nc=45

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

...

                   all        180       1080      0.997      0.996      0.995      0.995



80 epochs completed in 0.254 hours.
Optimizer stripped from /content/drive/MyDrive/celeba_dataset/CompositeDataset/runs/yolov8n_grid3x2/weights/last.pt, 6.3MB
Optimizer stripped from /content/drive/MyDrive/celeba_dataset/CompositeDataset/runs/yolov8n_grid3x2/weights/best.pt, 6.3MB

Validating /content/drive/MyDrive/celeba_dataset/CompositeDataset/runs/yolov8n_grid3x2/weights/best.pt...
Ultralytics 8.3.34 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 168 layers, 3,014,423 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  3.41it/s]


                   all        180       1080      0.997      0.996      0.995      0.995
                   228         23         23      0.997          1      0.995      0.995
                   447         19         19      0.996          1      0.995      0.995
                   487         24         24      0.993          1      0.995      0.995
                   800         22         22      0.997          1      0.995      0.995
                  1158         23         23      0.997          1      0.995      0.995
                  1499         26         26      0.998          1      0.995      0.995
                  1852         26         26      0.997          1      0.995      0.995
                  1964         19         19      0.997          1      0.995      0.995
                  2425         22         22      0.998          1      0.995      0.995
                  2463         26         26      0.997          1      0.995      0.995
                  252

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e6a6835a330>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,

In [10]:
from ultralytics import YOLO
import pandas as pd

# Load best weights
runs_dir = f"{COMP_ROOT}/runs/yolov8n_grid3x2/weights"
best = f"{runs_dir}/best.pt"

predictor = YOLO(best)
test_images = sorted(glob.glob(f"{IMAGES_DIR}/test/*.jpg"))[:12]  # a few samples

results = predictor.predict(
    source=test_images,
    imgsz=640,
    conf=0.25,
    save=True,                      # saves images with boxes
    project=f"{COMP_ROOT}/preds",
    name="test_samples",
    verbose=False
)

# Pretty print: Celeb ID, conf, bbox (xyxy)
for r, img_path in zip(results, test_images):
    print(f"\nImage: {Path(img_path).name}")
    if r.boxes is None or len(r.boxes) == 0:
        print("  No detections.")
        continue
    ids = r.boxes.cls.cpu().numpy().astype(int)
    confs = r.boxes.conf.cpu().numpy()
    xyxy = r.boxes.xyxy.cpu().numpy()

    rows = []
    for i, c in enumerate(ids):
        celeb_id_str = data_yaml["names"][c]   # true celeb ID string
        x1,y1,x2,y2 = xyxy[i]
        rows.append([celeb_id_str, float(confs[i]), (int(x1),int(y1),int(x2),int(y2))])
    df = pd.DataFrame(rows, columns=["Celeb ID","Confidence","Bbox (x_min,y_min,x_max,y_max)"])
    display(df)

Results saved to /content/drive/MyDrive/celeba_dataset/CompositeDataset/preds/test_samples

Image: comp_001020.jpg


,Celeb ID,Confidence,"Bbox (x_min,y_min,x_max,y_max)"
0,8656,0.992752,"(21, 375, 201, 595)"
1,10173,0.992404,"(447, 375, 626, 594)"
2,8722,0.992192,"(20, 56, 200, 275)"
3,3431,0.991417,"(447, 55, 627, 274)"
4,3401,0.989803,"(234, 56, 413, 275)"
5,9152,0.988730,"(234, 375, 413, 596)"



Image: comp_001021.jpg


,Celeb ID,Confidence,"Bbox (x_min,y_min,x_max,y_max)"
0,3699,0.996051,"(21, 55, 200, 275)"
1,8871,0.994985,"(22, 375, 200, 595)"
2,2562,0.994374,"(235, 375, 413, 595)"
3,7282,0.993868,"(447, 55, 626, 274)"
4,9152,0.991091,"(448, 375, 626, 594)"
5,2820,0.986760,"(234, 55, 413, 275)"



Image: comp_001022.jpg


...

,Celeb ID,Confidence,"Bbox (x_min,y_min,x_max,y_max)"
0,6098,0.995045,"(21, 55, 201, 275)"
1,487,0.992485,"(447, 56, 626, 275)"
2,8871,0.992189,"(234, 375, 413, 595)"
3,2837,0.992163,"(21, 375, 201, 595)"
4,7904,0.989955,"(234, 55, 414, 275)"
5,2425,0.989646,"(447, 375, 626, 594)"



Image: comp_001030.jpg


,Celeb ID,Confidence,"Bbox (x_min,y_min,x_max,y_max)"
0,1964,0.996367,"(22, 56, 199, 275)"
1,487,0.994957,"(21, 376, 201, 595)"
2,8968,0.994597,"(447, 375, 625, 595)"
3,4304,0.992683,"(234, 376, 413, 595)"
4,9256,0.990269,"(447, 56, 625, 275)"
5,228,0.988177,"(234, 55, 413, 275)"



Image: comp_001031.jpg


,Celeb ID,Confidence,"Bbox (x_min,y_min,x_max,y_max)"
0,9256,0.997301,"(447, 56, 627, 274)"
1,3321,0.995518,"(234, 55, 413, 274)"
2,2837,0.993487,"(234, 375, 413, 595)"
3,2562,0.990140,"(447, 375, 626, 594)"
4,3227,0.990060,"(21, 56, 201, 275)"
5,3401,0.989539,"(21, 375, 200, 595)"
